In [9]:
import pandas as pd
import numpy as np
import pickle
import os
from datetime import datetime
from sklearn.metrics import accuracy_score
from scipy.stats import ks_2samp
from sklearn.metrics.pairwise import cosine_similarity
import warnings
warnings.filterwarnings('ignore')

class TestDataDriftAnalyzer:
    def __init__(self, config_path="config.yaml"):
        self.config = self.load_config(config_path)
        self.test_df = None
        self.chunk_models = {}
        self.drift_results = {}
        self.vectorizer = None
        self.label_encoder = None
        
    def load_config(self, config_path):
        """Load configuration"""
        try:
            import yaml
            with open(config_path, 'r') as file:
                return yaml.safe_load(file)
        except:
            return self.get_default_config()
    
    def get_default_config(self):
        """Default configuration"""
        return {
            'paths': {
                'test_dataset': "balanced_dataset/test_data.csv",
                'models_dir': "models",
                'drift_reports_dir': "drift_reports",
                'drift_file': "test_data_drift_report.pkl"
            },
            'drift_detection': {
                'data_drift_threshold': 0.01,
                'concept_drift_threshold': 0.006,
                'significance_level': 0.006
            },
            'models': ['logistic_regression', 'decision_tree', 'neural_network', 'svm', 'lasso_regression']
        }
    
    def load_test_data(self):
        """Load test dataset"""
        test_path = self.config['paths']['test_dataset']
        possible_paths = [
            test_path,
            os.path.join("balanced_dataset", "test_data.csv"),
            os.path.join("balanced_dataset", "test_data.pkl"),
            "test_data.csv"
        ]
        
        for data_path in possible_paths:
            try:
                if os.path.exists(data_path):
                    if data_path.endswith('.pkl'):
                        with open(data_path, 'rb') as f:
                            self.test_df = pickle.load(f)
                    else:
                        self.test_df = pd.read_csv(data_path)
                    
                    print(f"Loaded test dataset from: {data_path}")
                    print(f"Test dataset shape: {self.test_df.shape}")
                    return True
            except Exception as e:
                print(f"Failed to load from {data_path}: {e}")
                continue
        
        print("Could not load test dataset")
        return False
    
    def auto_detect_columns(self):
        """Auto-detect text and label columns"""
        if self.test_df is None:
            return None, None
        
        # Label column candidates
        label_candidates = ['label', 'category', 'class', 'target', 'industry', 'industry_name']
        for candidate in label_candidates:
            if candidate in self.test_df.columns:
                label_col = candidate
                break
        else:
            for col in self.test_df.columns:
                if self.test_df[col].dtype == 'object' and 2 <= self.test_df[col].nunique() <= 50:
                    label_col = col
                    break
            else:
                label_col = self.test_df.columns[0] if len(self.test_df.columns) > 0 else None
        
        # Text column candidates
        text_candidates = ['text', 'content', 'summary', 'description', 'full_summary']
        for candidate in text_candidates:
            if candidate in self.test_df.columns:
                text_col = candidate
                break
        else:
            for col in self.test_df.columns:
                if self.test_df[col].dtype == 'object':
                    avg_length = self.test_df[col].astype(str).str.len().mean()
                    if avg_length > 10:
                        text_col = col
                        break
            else:
                text_col = self.test_df.columns[0] if len(self.test_df.columns) > 0 else None
        
        return text_col, label_col
    
    def load_chunk_models(self):
        """Load all chunk models from models directory"""
        models_dir = self.config['paths']['models_dir']
        models = self.config['models']
        
        print("Loading chunk models...")
        
        for model_name in models:
            self.chunk_models[model_name] = {}
            model_dir = os.path.join(models_dir, model_name)
            
            if not os.path.exists(model_dir):
                print(f"Model directory not found: {model_dir}")
                continue
            
            # Load all chunk model files
            for filename in os.listdir(model_dir):
                if filename.startswith('chunk_') and filename.endswith('.pkl'):
                    try:
                        chunk_id = filename.replace('chunk_', '').replace('.pkl', '')
                        filepath = os.path.join(model_dir, filename)
                        
                        with open(filepath, 'rb') as f:
                            model_data = pickle.load(f)
                        
                        self.chunk_models[model_name][chunk_id] = model_data
                        print(f"Loaded {model_name} chunk {chunk_id}")
                        
                    except Exception as e:
                        print(f"Error loading {model_name} {filename}: {e}")
        
        # Print summary
        for model_name, chunks in self.chunk_models.items():
            print(f"{model_name}: {len(chunks)} chunks loaded")
        
        return self.chunk_models
    
    def prepare_test_features(self, vectorizer, label_encoder):
        """Prepare test features using the same vectorizer and label encoder"""
        text_col, label_col = self.auto_detect_columns()
        
        if not text_col or not label_col:
            print("Could not auto-detect text or label columns")
            print(f"Available columns: {self.test_df.columns.tolist()}")
            return False, None, None
        
        print(f"Using text column: '{text_col}'")
        print(f"Using label column: '{label_col}'")
        
        # Prepare text features
        texts = self.test_df[text_col].fillna('').astype(str)
        X_test = vectorizer.transform(texts)
        
        # Prepare labels
        y_test = self.test_df[label_col]
        y_test_encoded = label_encoder.transform(y_test)
        
        print(f"Test features shape: {X_test.shape}")
        print(f"Test labels shape: {y_test_encoded.shape}")
        
        return True, X_test, y_test_encoded
    
    def find_vectorizer_and_encoder(self):
        """Find vectorizer and label encoder from model files"""
        models_dir = self.config['paths']['models_dir']
        
        # Check each model directory for files containing vectorizer and encoder
        for model_name in self.config['models']:
            model_dir = os.path.join(models_dir, model_name)
            if not os.path.exists(model_dir):
                continue
            
            for filename in os.listdir(model_dir):
                if filename.endswith('.pkl'):
                    try:
                        filepath = os.path.join(model_dir, filename)
                        with open(filepath, 'rb') as f:
                            model_data = pickle.load(f)
                        
                        # Check if this file contains vectorizer and encoder
                        if 'vectorizer' in model_data and 'label_encoder' in model_data:
                            self.vectorizer = model_data['vectorizer']
                            self.label_encoder = model_data['label_encoder']
                            print(f"Found vectorizer and label encoder in {model_name}/{filename}")
                            return True
                            
                    except Exception as e:
                        continue
        
        # If not found, try to load from overall best model
        best_model_path = os.path.join("best_models", "overall_best_model.pkl")
        if os.path.exists(best_model_path):
            try:
                with open(best_model_path, 'rb') as f:
                    best_model_data = pickle.load(f)
                if 'vectorizer' in best_model_data and 'label_encoder' in best_model_data:
                    self.vectorizer = best_model_data['vectorizer']
                    self.label_encoder = best_model_data['label_encoder']
                    print("Found vectorizer and label encoder in overall best model")
                    return True
            except Exception as e:
                print(f"Error loading overall best model: {e}")
        
        print("Could not find vectorizer and label encoder")
        return False
    
    def calculate_data_drift(self, X_train_chunk, X_test):
        """Calculate data drift between training chunk and test data"""
        try:
            # Convert to dense arrays if sparse
            X1 = X_train_chunk.toarray() if hasattr(X_train_chunk, 'toarray') else X_train_chunk
            X2 = X_test.toarray() if hasattr(X_test, 'toarray') else X_test
            
            # Sample features for efficiency
            n_features = min(X1.shape[1], X2.shape[1], 100)
            feature_indices = np.random.choice(min(X1.shape[1], X2.shape[1]), n_features, replace=False)
            
            # Calculate KS scores
            ks_scores = []
            for feature_idx in feature_indices:
                if feature_idx < X1.shape[1] and feature_idx < X2.shape[1]:
                    ks_stat, _ = ks_2samp(X1[:, feature_idx], X2[:, feature_idx])
                    ks_scores.append(ks_stat)
            
            avg_ks_score = np.mean(ks_scores) if ks_scores else 0
            
            # Calculate cosine distance
            mean1 = np.mean(X1, axis=0)
            mean2 = np.mean(X2, axis=0)
            cosine_sim = cosine_similarity([mean1], [mean2])[0][0]
            cosine_distance = 1 - cosine_sim
            
            # Combined drift score
            data_drift_score = (avg_ks_score + cosine_distance) / 2
            
            drift_threshold = self.config['drift_detection']['data_drift_threshold']
            
            return {
                'data_drift_score': float(data_drift_score),
                'ks_score': float(avg_ks_score),
                'cosine_distance': float(cosine_distance),
                'has_drift': data_drift_score > drift_threshold
            }
            
        except Exception as e:
            print(f"Error calculating data drift: {e}")
            return {
                'data_drift_score': 1.0,
                'ks_score': 1.0,
                'cosine_distance': 1.0,
                'has_drift': True,
                'error': str(e)
            }
    
    def calculate_concept_drift(self, model, X_train_chunk, y_train_chunk, X_test, y_test):
        """Calculate concept drift using model performance"""
        try:
            # Train model on chunk data
            model_copy = model.fit(X_train_chunk, y_train_chunk)
            
            # Calculate accuracy on training chunk and test data
            train_accuracy = accuracy_score(y_train_chunk, model_copy.predict(X_train_chunk))
            test_accuracy = accuracy_score(y_test, model_copy.predict(X_test))
            
            accuracy_drop = train_accuracy - test_accuracy
            concept_drift_score = max(0, accuracy_drop)
            
            drift_threshold = self.config['drift_detection']['concept_drift_threshold']
            
            return {
                'concept_drift_score': float(concept_drift_score),
                'accuracy_drop': float(accuracy_drop),
                'train_accuracy': float(train_accuracy),
                'test_accuracy': float(test_accuracy),
                'has_drift': concept_drift_score > drift_threshold
            }
            
        except Exception as e:
            print(f"Error calculating concept drift: {e}")
            return {
                'concept_drift_score': 1.0,
                'accuracy_drop': 1.0,
                'train_accuracy': 0.0,
                'test_accuracy': 0.0,
                'has_drift': True,
                'error': str(e)
            }
    
    def analyze_test_data_drift(self):
        """Main method to analyze drift between chunk models and test data"""
        print("="*60)
        print("ANALYZING DRIFT BETWEEN CHUNK MODELS AND TEST DATA")
        print("="*60)
        
        # Load test data
        if not self.load_test_data():
            return False
        
        # Load chunk models
        self.load_chunk_models()
        
        # Find vectorizer and label encoder
        if not self.find_vectorizer_and_encoder():
            print("Need vectorizer and label encoder to process test data")
            return False
        
        # Prepare test features
        success, X_test, y_test = self.prepare_test_features(self.vectorizer, self.label_encoder)
        if not success:
            return False
        
        # Initialize drift results structure
        self.drift_results = {
            'data_drift': {},
            'concept_drift': {},
            'summary': {
                'analysis_timestamp': datetime.now().isoformat(),
                'test_data_size': len(self.test_df),
                'total_chunks_analyzed': 0,
                'drift_thresholds': self.config['drift_detection']
            }
        }
        
        print(f"\nTest data size: {len(self.test_df)}")
        print(f"Number of features: {X_test.shape[1]}")
        print(f"Number of classes: {len(np.unique(y_test))}")
        
        # Analyze drift for each model and chunk
        data_drift_scores = []
        concept_drift_scores_by_model = {model: [] for model in self.config['models']}
        
        for model_name, chunks in self.chunk_models.items():
            print(f"\nAnalyzing {model_name}...")
            
            for chunk_id, chunk_data in chunks.items():
                print(f"  Chunk {chunk_id}: ", end="")
                
                # Get training data from chunk (if available)
                if 'X' in chunk_data and 'y' in chunk_data:
                    X_train_chunk = chunk_data['X']
                    y_train_chunk = chunk_data['y']
                else:
                    # If chunk doesn't contain training data, skip data drift
                    print("No training data in chunk, skipping data drift")
                    X_train_chunk = None
                    y_train_chunk = None
                
                # Calculate data drift
                if X_train_chunk is not None:
                    data_drift = self.calculate_data_drift(X_train_chunk, X_test)
                    drift_key = f"{model_name}_chunk_{chunk_id}_to_test"
                    self.drift_results['data_drift'][drift_key] = data_drift
                    data_drift_scores.append(data_drift['data_drift_score'])
                    
                    data_status = "DRIFT" if data_drift['has_drift'] else "OK"
                    print(f"Data: {data_drift['data_drift_score']:.4f} ({data_status})", end="")
                else:
                    print("Data: N/A", end="")
                
                # Calculate concept drift
                if 'model' in chunk_data and y_train_chunk is not None:
                    concept_drift = self.calculate_concept_drift(
                        chunk_data['model'], X_train_chunk, y_train_chunk, X_test, y_test
                    )
                    drift_key = f"{model_name}_chunk_{chunk_id}_to_test"
                    self.drift_results['concept_drift'][drift_key] = concept_drift
                    concept_drift_scores_by_model[model_name].append(concept_drift['concept_drift_score'])
                    
                    concept_status = "DRIFT" if concept_drift['has_drift'] else "OK"
                    print(f" | Concept: {concept_drift['concept_drift_score']:.4f} ({concept_status})")
                else:
                    print(" | Concept: N/A")
        
        # Calculate summaries
        if data_drift_scores:
            self.drift_results['summary']['data_drift'] = {
                'mean_score': float(np.mean(data_drift_scores)),
                'max_score': float(np.max(data_drift_scores)),
                'min_score': float(np.min(data_drift_scores)),
                'drift_detected_count': sum(1 for score in data_drift_scores 
                                          if score > self.config['drift_detection']['data_drift_threshold']),
                'total_comparisons': len(data_drift_scores)
            }
        
        # Model-wise concept drift summaries
        for model_name, scores in concept_drift_scores_by_model.items():
            if scores:
                self.drift_results['concept_drift'][f"{model_name}_summary"] = {
                    'mean_concept_drift': float(np.mean(scores)),
                    'max_concept_drift': float(np.max(scores)),
                    'min_concept_drift': float(np.min(scores)),
                    'drift_detected_count': sum(1 for score in scores 
                                              if score > self.config['drift_detection']['concept_drift_threshold']),
                    'chunks_analyzed': len(scores)
                }
        
        self.drift_results['summary']['total_chunks_analyzed'] = sum(
            len(chunks) for chunks in self.chunk_models.values()
        )
        
        return True
    
    def print_drift_summary(self):
        """Print summary of drift analysis"""
        if not self.drift_results:
            print("No drift results available")
            return
        
        print("\n" + "="*70)
        print("DRIFT ANALYSIS SUMMARY - TEST DATA")
        print("="*70)
        
        summary = self.drift_results['summary']
        
        print(f"Analysis Timestamp: {summary['analysis_timestamp']}")
        print(f"Test Data Size: {summary['test_data_size']}")
        print(f"Total Chunks Analyzed: {summary['total_chunks_analyzed']}")
        
        if 'data_drift' in summary:
            data_summary = summary['data_drift']
            print(f"\nDATA DRIFT SUMMARY:")
            print(f"  Mean Data Drift Score: {data_summary['mean_score']:.4f}")
            print(f"  Max Data Drift Score:  {data_summary['max_score']:.4f}")
            print(f"  Min Data Drift Score:  {data_summary['min_score']:.4f}")
            print(f"  Drift Detected in:     {data_summary['drift_detected_count']} out of {data_summary['total_comparisons']} comparisons")
        
        print(f"\nCONCEPT DRIFT SUMMARY BY MODEL:")
        for model_name in self.config['models']:
            model_key = f"{model_name}_summary"
            if model_key in self.drift_results['concept_drift']:
                model_summary = self.drift_results['concept_drift'][model_key]
                print(f"  {model_name:20} | Mean: {model_summary['mean_concept_drift']:.4f} | "
                      f"Max: {model_summary['max_concept_drift']:.4f} | "
                      f"Drifts: {model_summary['drift_detected_count']}/{model_summary['chunks_analyzed']}")
    
    def save_drift_results(self):
        """Save drift results to pickle file"""
        drift_reports_dir = self.config['paths']['drift_reports_dir']
        os.makedirs(drift_reports_dir, exist_ok=True)
        
        drift_file = os.path.join(drift_reports_dir, self.config['paths']['drift_file'])
        
        with open(drift_file, 'wb') as f:
            pickle.dump(self.drift_results, f)
        
        print(f"\nDrift results saved to: {drift_file}")
        return drift_file
    
    def run_analysis(self):
        """Run complete drift analysis"""
        print("STARTING TEST DATA DRIFT ANALYSIS")
        print("=" * 50)
        
        success = self.analyze_test_data_drift()
        
        if success:
            self.print_drift_summary()
            self.save_drift_results()
            print("\nANALYSIS COMPLETED SUCCESSFULLY")
        else:
            print("\nANALYSIS FAILED")
        
        return success


def main():
    analyzer = TestDataDriftAnalyzer("config.yaml")
    analyzer.run_analysis()


if __name__ == "__main__":
    main()

STARTING TEST DATA DRIFT ANALYSIS
ANALYZING DRIFT BETWEEN CHUNK MODELS AND TEST DATA
Loaded test dataset from: train_test_data/test_data.csv
Test dataset shape: (55, 12)
Loading chunk models...
Loaded logistic_regression chunk 08
Loaded logistic_regression chunk 9
Loaded logistic_regression chunk 4
Loaded logistic_regression chunk 04
Loaded logistic_regression chunk 05
Loaded logistic_regression chunk 5
Loaded logistic_regression chunk 15
Loaded logistic_regression chunk 3
Loaded logistic_regression chunk 06
Loaded logistic_regression chunk 8
Loaded logistic_regression chunk 09
Loaded logistic_regression chunk 13
Loaded logistic_regression chunk 2
Loaded logistic_regression chunk 07
Loaded logistic_regression chunk 03
Loaded logistic_regression chunk 10
Loaded logistic_regression chunk 11
Loaded logistic_regression chunk 14
Loaded logistic_regression chunk 01
Loaded logistic_regression chunk 1
Loaded logistic_regression chunk 12
Loaded logistic_regression chunk 6
Loaded logistic_regres